# 🇲🇬 Voambolana Malagasy — NLP Project

Ce notebook centralise la collecte de données et la préparation du modèle IA.

### 📑 Étapes d'utilisation :
1.  **Exécutez la cellule de configuration** ci-dessous pour connecter votre Drive et récupérer le code.
2.  **Lancez les scrapers** (Phases 1-4) pour collecter les paroles.
3.  **Nettoyez et Consolidez** (Phases 5-6) pour obtenir un corpus propre.
4.  **Fondations NLP** (Phase 7-8) : Tokenizer et Embeddings.

## 🛠️ 1. Configuration (Drive + Git)

In [ ]:
from google.colab import drive
import os

# 1. Monter le Drive
print("🔗 Connexion à Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Drive monté avec succès !")
except Exception as e:
    print("❌ Erreur lors du montage du Drive.")

# 2. Se placer dans le dossier du projet
REPO_NAME = "nlp_malagasy"
BASE_PATH = "/content/drive/MyDrive/" + REPO_NAME

if not os.path.exists(BASE_PATH):
    %cd /content/drive/MyDrive
    !git clone https://github.com/franck504/nlp_malagasy.git

%cd $BASE_PATH
!git pull origin main

print(f"\n📍 Position actuelle : {os.getcwd()}")
!ls

# 🏗️ PARTIE 1 : COLLECTE DES DONNÉES (Scraping)

### 🎵 Phase 1-3 : Discovery, Scraping & Stats
Exécutez ces étapes dans l'ordre pour collecter toutes les chansons.

In [ ]:
!pip install requests beautifulsoup4
%cd $BASE_PATH/tononkira_rehetra

print("--- PHASE 1 : DISCOVERY ---")
!python 01_discover_artists.py --delay 1.0

print("\n--- PHASE 2 : SCRAPING (TURBO) ---")
!python 02_scrape_lyrics.py --delay 0.5 --artist-workers 4 --song-workers 10

print("\n--- PHASE 3 : STATISTIQUES ---")
!python 03_stats.py

### 📂 Phase 4 : Fusion du Corpus Brut
Fusionne tous les fichiers `.txt` individuels.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 04_merge_corpus.py

# 🧹 PARTIE 2 : NETTOYAGE ET CONSOLIDATION

### 🧼 Phase 5 : Purification (Nettoyage de Langue)
Supprime les blocs en Français/Anglais pour ne garder que le Malgache pur.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 05_clean_corpus.py --input malagasy_lyrics_corpus.txt --output malagasy_lyrics_cleaned.txt

### 📖 Phase 4.5 : Consolidation (Lyrics + Bible)
Fusionne les chansons avec la Bible et supprime les répétitions.

In [ ]:
%cd $BASE_PATH/tononkira_rehetra
!python 06_consolidate_corpus.py --lyrics malagasy_lyrics_cleaned.txt --bible ../from_bible_json --output malagasy_corpus_v1_fixed.txt

# 🧠 PARTIE 3 : FONDATIONS NLP

### ✂️ Phase 7 : Tokenisation (Apprentissage des Patterns)
Apprend à l'IA comment découper les mots malgaches (tovona, tovana, etc.).

In [ ]:
!pip install tokenizers
%cd $BASE_PATH/tononkira_rehetra
!python 07_train_tokenizer.py --corpus malagasy_corpus_v1_fixed.txt --output tokenizer_mg

### 📐 Phase 8 : Embeddings (Espace des Sens)
Crée les vecteurs mathématiques pour chaque mot afin de comprendre les synonymes et les contextes.

In [ ]:
!pip install gensim
%cd $BASE_PATH/tononkira_rehetra
!python 08_train_embeddings.py --corpus malagasy_corpus_v1_fixed.txt --output embeddings_mg